<a href="https://colab.research.google.com/github/WHU-Peter/COMP6200-Project/blob/main/Look_up_Table_Block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
def softmax_temperature(logits, temperature):
    pro = F.softmax(logits / temperature, dim=-1)
    return pro;

In [ ]:
def mapping_onehot_vector(vector):
    return vector @ torch.LongTensor(range(0, vector.shape[1]))

In [ ]:
class LearnableLookUpTable(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, init_temperature: float):
        super(LearnableLookUpTable, self).__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.temperature = nn.Parameter(torch.tensor(init_temperature, requires_grad=True))

    def forward(self, x):
        if self.training:
          x = softmax_temperature(x, self.temperature)
          return x @ self.emb.weight
        else:
          x = softmax_temperature(x, 0.00001)
          
          return self.emb(x)

In [ ]:
class AnnealingLookUpTable(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        super(AnnealingLookUpTable, self).__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x, temperature):
        if self.training:
          x = self.softmax_temperature(x, temperature)
          return x @ self.emb.weight
        else:
          x = self.softmax_temperature(x, 0.00001)
          x = x @ torch.LongTensor(range(0, x.shape[1]))
          return self.emb(x)